# Graph Structural Analysis

This notebook performs structural analysis on the cyber incidents graph including:
- Triad analysis (transitive, cyclic, fan-in, fan-out)
- Clustering analysis
- Cliques and maximal cliques
- In-k-core, out-k-core, and onion structure
- Ego networks (in and out)
- Community detection (Louvain and Girvan-Newman)

In [ ]:
# Cell 1: Import delle librerie necessarie
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings

# Import per community detection
try:
    import community.community_louvain as community_louvain
except ImportError:
    import community as community_louvain

from networkx.algorithms.community import girvan_newman
from networkx.algorithms.community.quality import modularity

# Configurazione visualizzazione
plt.rcParams['figure.figsize'] = (14, 10)
sns.set_style('whitegrid')
warnings.filterwarnings('ignore')

print("Librerie importate correttamente!")

In [ ]:
# Cell 2: Caricamento del file GML
file_path = '../data/cyber_incidents_graph.gml'
output_folder = './grafici/'

import os
os.makedirs(output_folder, exist_ok=True)

try:
    G = nx.read_gml(file_path)
    print(f"Grafo caricato correttamente da: {file_path}")
    print(f"\n=== INFORMAZIONI SUL GRAFO ===")
    print(f"Numero di nodi: {G.number_of_nodes()}")
    print(f"Numero di archi: {G.number_of_edges()}")
    print(f"Tipo di grafo: {'Diretto' if G.is_directed() else 'Non diretto'}")
    print(f"Grafo pesato: {'Si' if nx.is_weighted(G) else 'No'}")
    print(f"\nNodi del grafo:")
    print(list(G.nodes())[:10], "..." if G.number_of_nodes() > 10 else "")
    
except FileNotFoundError:
    print(f"ERRORE: Il file {file_path} non e stato trovato.")
except Exception as e:
    print(f"ERRORE durante il caricamento del grafo: {e}")

In [ ]:
# Cell 3: Analisi delle Triadi (Transitive, Cicliche, Fan-in, Fan-out)
print("=" * 60)
print("ANALISI DELLE TRIADI")
print("=" * 60)

# Calcolo del censimento delle triadi
triad_census = nx.triadic_census(G)

# Descrizione dei tipi di triadi
triad_descriptions = {
    '003': 'Vuota (nessun arco)',
    '012': 'Un singolo arco',
    '102': 'Arco reciproco',
    '021D': 'Fan-out (D divergente)',
    '021U': 'Fan-in (U convergente)',
    '021C': 'Catena (C)',
    '111D': 'Arco reciproco + fan-out',
    '111U': 'Arco reciproco + fan-in',
    '030T': 'Transitiva',
    '030C': 'Ciclica',
    '201': 'Due archi reciproci',
    '120D': 'Fan-out con arco reciproco',
    '120U': 'Fan-in con arco reciproco',
    '120C': 'Catena con arco reciproco',
    '210': 'Quasi completa',
    '300': 'Completa (tutti archi reciproci)'
}

# Stampa risultati testuali
print("\n--- Censimento Triadi ---")
df_triads = pd.DataFrame([
    {'Tipo': k, 'Descrizione': triad_descriptions.get(k, 'N/A'), 'Conteggio': v}
    for k, v in triad_census.items()
])
df_triads = df_triads.sort_values('Conteggio', ascending=False)
print(df_triads.to_string(index=False))

# Identificazione triadi specifiche
print("\n--- Triadi Specifiche ---")
print(f"Triadi Transitive (030T): {triad_census.get('030T', 0)}")
print(f"Triadi Cicliche (030C): {triad_census.get('030C', 0)}")
print(f"Triadi Fan-in (021U): {triad_census.get('021U', 0)}")
print(f"Triadi Fan-out (021D): {triad_census.get('021D', 0)}")

# Calcolo transitività
transitivity = nx.transitivity(G)
print(f"\nTransitivita del grafo: {transitivity:.4f}")

# Visualizzazione grafica
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Grafico 1: Bar chart di tutte le triadi
df_triads_nonzero = df_triads[df_triads['Conteggio'] > 0]
ax1 = axes[0]
bars = ax1.barh(df_triads_nonzero['Tipo'], df_triads_nonzero['Conteggio'], color=plt.cm.viridis(np.linspace(0, 1, len(df_triads_nonzero))))
ax1.set_xlabel('Conteggio')
ax1.set_ylabel('Tipo di Triade')
ax1.set_title('Distribuzione dei Tipi di Triadi')
for bar, count in zip(bars, df_triads_nonzero['Conteggio']):
    ax1.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2, str(count), va='center', fontsize=9)

# Grafico 2: Pie chart delle triadi chiave
ax2 = axes[1]
key_triads = {'Transitive (030T)': triad_census.get('030T', 0),
              'Cicliche (030C)': triad_census.get('030C', 0),
              'Fan-in (021U)': triad_census.get('021U', 0),
              'Fan-out (021D)': triad_census.get('021D', 0),
              'Catena (021C)': triad_census.get('021C', 0)}
key_triads = {k: v for k, v in key_triads.items() if v > 0}
if key_triads:
    colors = plt.cm.Set3(np.linspace(0, 1, len(key_triads)))
    wedges, texts, autotexts = ax2.pie(key_triads.values(), labels=key_triads.keys(), autopct='%1.1f%%', colors=colors)
    ax2.set_title('Distribuzione Triadi Chiave')
else:
    ax2.text(0.5, 0.5, 'Nessuna triade chiave trovata', ha='center', va='center')

plt.tight_layout()
plt.savefig(f"{output_folder}/triad_analysis.png", dpi=150, bbox_inches='tight')
plt.show()

# Esempio visuale di una triade nel grafo
print("\n--- Esempio di Triadi nel Grafo ---")
triangles = [clique for clique in nx.enumerate_all_cliques(G.to_undirected()) if len(clique) == 3]
if triangles:
    print(f"Numero di triangoli (triadi connesse): {len(triangles)}")
    print(f"Primi 5 triangoli: {triangles[:5]}")
else:
    print("Nessun triangolo trovato nel grafo.")

In [ ]:
# Cell 4: Analisi di Clustering
print("=" * 60)
print("ANALISI DI CLUSTERING")
print("=" * 60)

# Coefficiente di clustering locale per ogni nodo
clustering_coeffs = nx.clustering(G)

# Coefficiente di clustering medio
avg_clustering = nx.average_clustering(G)

# Coefficiente di clustering globale (transitività)
global_clustering = nx.transitivity(G)

print("\n--- Metriche di Clustering ---")
print(f"Coefficiente di clustering medio: {avg_clustering:.4f}")
print(f"Coefficiente di clustering globale (transitività): {global_clustering:.4f}")

# DataFrame con coefficienti per nodo
df_clustering = pd.DataFrame([
    {'Nodo': node, 'Clustering': coeff}
    for node, coeff in clustering_coeffs.items()
])
df_clustering = df_clustering.sort_values('Clustering', ascending=False)

print("\n--- Top 15 Nodi per Coefficiente di Clustering ---")
print(df_clustering.head(15).to_string(index=False))

# Statistiche descrittive
print("\n--- Statistiche Descrittive del Clustering ---")
print(df_clustering['Clustering'].describe())

# Visualizzazione grafica
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Grafico 1: Distribuzione dei coefficienti di clustering
ax1 = axes[0, 0]
ax1.hist(df_clustering['Clustering'], bins=20, color='steelblue', edgecolor='black', alpha=0.7)
ax1.axvline(avg_clustering, color='red', linestyle='--', linewidth=2, label=f'Media: {avg_clustering:.4f}')
ax1.set_xlabel('Coefficiente di Clustering')
ax1.set_ylabel('Frequenza')
ax1.set_title('Distribuzione dei Coefficienti di Clustering')
ax1.legend()

# Grafico 2: Top 20 nodi per clustering
ax2 = axes[0, 1]
top_20_clustering = df_clustering.head(20)
colors = plt.cm.Blues(np.linspace(0.3, 1, len(top_20_clustering)))
ax2.barh(top_20_clustering['Nodo'], top_20_clustering['Clustering'], color=colors)
ax2.set_xlabel('Coefficiente di Clustering')
ax2.set_ylabel('Nodo')
ax2.set_title('Top 20 Nodi per Coefficiente di Clustering')

# Grafico 3: Network con nodi colorati per clustering
ax3 = axes[1, 0]
pos = nx.spring_layout(G, seed=42, k=2)
node_colors = [clustering_coeffs.get(node, 0) for node in G.nodes()]
nodes = nx.draw_networkx_nodes(G, pos, node_color=node_colors, cmap=plt.cm.YlOrRd, 
                                node_size=500, alpha=0.8, ax=ax3)
nx.draw_networkx_edges(G, pos, alpha=0.3, ax=ax3)
nx.draw_networkx_labels(G, pos, font_size=7, ax=ax3)
plt.colorbar(nodes, ax=ax3, label='Coefficiente di Clustering')
ax3.set_title('Network con Clustering Coefficient')
ax3.axis('off')

# Grafico 4: Scatter plot Grado vs Clustering
ax4 = axes[1, 1]
degrees = dict(G.degree())
degree_list = [degrees[node] for node in df_clustering['Nodo']]
ax4.scatter(degree_list, df_clustering['Clustering'], alpha=0.6, c='purple', s=100)
ax4.set_xlabel('Grado del Nodo')
ax4.set_ylabel('Coefficiente di Clustering')
ax4.set_title('Grado vs Coefficiente di Clustering')

plt.tight_layout()
plt.savefig(f"{output_folder}/clustering_analysis.png", dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Cell 5: Analisi delle Clique e Clique Massima
print("=" * 60)
print("ANALISI DELLE CLIQUE")
print("=" * 60)

# Per l'analisi delle clique, convertiamo in grafo non diretto
G_undirected = G.to_undirected()

# Trova tutte le clique
all_cliques = list(nx.find_cliques(G_undirected))

print(f"\n--- Statistiche sulle Clique ---")
print(f"Numero totale di clique massimali: {len(all_cliques)}")

# Distribuzione delle dimensioni delle clique
clique_sizes = [len(c) for c in all_cliques]
size_distribution = Counter(clique_sizes)

print(f"\nDistribuzione delle dimensioni delle clique:")
for size, count in sorted(size_distribution.items()):
    print(f"  Dimensione {size}: {count} clique")

# Clique massima
max_clique = max(all_cliques, key=len)
print(f"\n--- Clique Massima ---")
print(f"Dimensione: {len(max_clique)}")
print(f"Nodi: {max_clique}")

# Numero di clique
clique_number = nx.graph_clique_number(G_undirected)
print(f"\nNumero di clique (dimensione massima): {clique_number}")

# Clique contenenti nodi specifici (es. i nodi con più connessioni)
top_nodes = sorted(dict(G.degree()).items(), key=lambda x: x[1], reverse=True)[:5]
print(f"\n--- Clique per i Top 5 Nodi ---")
for node, degree in top_nodes:
    node_cliques = [c for c in all_cliques if node in c]
    max_node_clique = max(node_cliques, key=len) if node_cliques else []
    print(f"{node} (grado {degree}): {len(node_cliques)} clique, max dimensione: {len(max_node_clique)}")

# Visualizzazione grafica
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Grafico 1: Distribuzione dimensioni clique
ax1 = axes[0]
sizes = sorted(size_distribution.keys())
counts = [size_distribution[s] for s in sizes]
ax1.bar(sizes, counts, color='teal', edgecolor='black')
ax1.set_xlabel('Dimensione della Clique')
ax1.set_ylabel('Numero di Clique')
ax1.set_title('Distribuzione delle Dimensioni delle Clique')
ax1.set_xticks(sizes)

# Grafico 2: Clique massima visualizzata
ax2 = axes[1]
subgraph_max = G_undirected.subgraph(max_clique)
pos_clique = nx.spring_layout(subgraph_max, seed=42)
nx.draw_networkx_nodes(subgraph_max, pos_clique, node_color='gold', node_size=800, ax=ax2)
nx.draw_networkx_edges(subgraph_max, pos_clique, width=2, ax=ax2)
nx.draw_networkx_labels(subgraph_max, pos_clique, font_size=10, font_weight='bold', ax=ax2)
ax2.set_title(f'Clique Massima (dimensione {len(max_clique)})')
ax2.axis('off')

# Grafico 3: Network con clique massima evidenziata
ax3 = axes[2]
pos = nx.spring_layout(G_undirected, seed=42, k=2)
node_colors = ['gold' if node in max_clique else 'lightblue' for node in G_undirected.nodes()]
node_sizes = [800 if node in max_clique else 300 for node in G_undirected.nodes()]
nx.draw_networkx_nodes(G_undirected, pos, node_color=node_colors, node_size=node_sizes, ax=ax3)
nx.draw_networkx_edges(G_undirected, pos, alpha=0.3, ax=ax3)
nx.draw_networkx_labels(G_undirected, pos, font_size=6, ax=ax3)
ax3.set_title('Network con Clique Massima Evidenziata')
ax3.axis('off')

plt.tight_layout()
plt.savefig(f"{output_folder}/clique_analysis.png", dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Cell 6: In-k-core, Out-k-core e Struttura a Cipolla
print("=" * 60)
print("ANALISI K-CORE E STRUTTURA A CIPOLLA")
print("=" * 60)

# Per i grafi diretti, calcoliamo in-degree e out-degree k-core
G_undirected = G.to_undirected()

# K-core standard (su grafo non diretto)
core_numbers = nx.core_number(G_undirected)
max_core = max(core_numbers.values())

print(f"\n--- K-Core Analysis ---")
print(f"Massimo k-core: {max_core}")

# Distribuzione dei core numbers
core_distribution = Counter(core_numbers.values())
print(f"\nDistribuzione dei Core Numbers:")
for k, count in sorted(core_distribution.items()):
    print(f"  k={k}: {count} nodi")

# In-k-core basato su in-degree
print(f"\n--- In-K-Core (basato su In-Degree) ---")
in_degrees = dict(G.in_degree())
in_core = {}
for k in range(1, max(in_degrees.values()) + 1):
    nodes_in_core = [n for n, d in in_degrees.items() if d >= k]
    if nodes_in_core:
        in_core[k] = nodes_in_core
        if k <= 5 or k == max(in_degrees.values()):
            print(f"  In-{k}-core: {len(nodes_in_core)} nodi")

# Out-k-core basato su out-degree
print(f"\n--- Out-K-Core (basato su Out-Degree) ---")
out_degrees = dict(G.out_degree())
out_core = {}
for k in range(1, max(out_degrees.values()) + 1):
    nodes_out_core = [n for n, d in out_degrees.items() if d >= k]
    if nodes_out_core:
        out_core[k] = nodes_out_core
        if k <= 5 or k == max(out_degrees.values()):
            print(f"  Out-{k}-core: {len(nodes_out_core)} nodi")

# Struttura a cipolla (onion decomposition)
print(f"\n--- Struttura a Cipolla (Onion Decomposition) ---")
onion_layers = nx.onion_layers(G_undirected)
layer_distribution = Counter(onion_layers.values())
print(f"Numero di strati: {max(onion_layers.values())}")
print(f"\nDistribuzione degli Strati:")
for layer, count in sorted(layer_distribution.items()):
    print(f"  Strato {layer}: {count} nodi")

# Nodi nel core più interno
innermost_core = nx.k_core(G_undirected, k=max_core)
print(f"\nNodi nel core più interno (k={max_core}): {list(innermost_core.nodes())}")

# Visualizzazione grafica
fig, axes = plt.subplots(2, 2, figsize=(16, 14))

# Grafico 1: Network colorato per core number
ax1 = axes[0, 0]
pos = nx.spring_layout(G_undirected, seed=42, k=2)
node_colors = [core_numbers[node] for node in G_undirected.nodes()]
nodes = nx.draw_networkx_nodes(G_undirected, pos, node_color=node_colors, cmap=plt.cm.plasma,
                                node_size=500, ax=ax1)
nx.draw_networkx_edges(G_undirected, pos, alpha=0.3, ax=ax1)
nx.draw_networkx_labels(G_undirected, pos, font_size=6, ax=ax1)
plt.colorbar(nodes, ax=ax1, label='Core Number')
ax1.set_title('Network colorato per K-Core Number')
ax1.axis('off')

# Grafico 2: Distribuzione K-Core
ax2 = axes[0, 1]
k_values = sorted(core_distribution.keys())
counts = [core_distribution[k] for k in k_values]
ax2.bar(k_values, counts, color='purple', edgecolor='black', alpha=0.7)
ax2.set_xlabel('Core Number (k)')
ax2.set_ylabel('Numero di Nodi')
ax2.set_title('Distribuzione dei K-Core Numbers')

# Grafico 3: Network colorato per onion layer
ax3 = axes[1, 0]
layer_colors = [onion_layers[node] for node in G_undirected.nodes()]
nodes = nx.draw_networkx_nodes(G_undirected, pos, node_color=layer_colors, cmap=plt.cm.Spectral,
                                node_size=500, ax=ax3)
nx.draw_networkx_edges(G_undirected, pos, alpha=0.3, ax=ax3)
nx.draw_networkx_labels(G_undirected, pos, font_size=6, ax=ax3)
plt.colorbar(nodes, ax=ax3, label='Onion Layer')
ax3.set_title('Network colorato per Strato Cipolla')
ax3.axis('off')

# Grafico 4: Confronto In-degree vs Out-degree
ax4 = axes[1, 1]
nodes_list = list(G.nodes())
in_deg = [in_degrees[n] for n in nodes_list]
out_deg = [out_degrees[n] for n in nodes_list]
ax4.scatter(in_deg, out_deg, alpha=0.6, c='orange', s=100)
ax4.set_xlabel('In-Degree')
ax4.set_ylabel('Out-Degree')
ax4.set_title('In-Degree vs Out-Degree')
# Aggiungi linea diagonale
max_deg = max(max(in_deg), max(out_deg))
ax4.plot([0, max_deg], [0, max_deg], 'r--', alpha=0.5, label='y=x')
ax4.legend()

plt.tight_layout()
plt.savefig(f"{output_folder}/kcore_onion_analysis.png", dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Cell 7: Ego Network Entranti e Uscenti
print("=" * 60)
print("ANALISI EGO NETWORK (ENTRANTI E USCENTI)")
print("=" * 60)

# Selezioniamo i nodi più importanti per l'analisi ego
in_degrees = dict(G.in_degree())
out_degrees = dict(G.out_degree())

# Nodi con più archi entranti (target principali)
top_in_nodes = sorted(in_degrees.items(), key=lambda x: x[1], reverse=True)[:5]
# Nodi con più archi uscenti (sorgenti principali)
top_out_nodes = sorted(out_degrees.items(), key=lambda x: x[1], reverse=True)[:5]

print("\n--- Top 5 Nodi per In-Degree (Target/Pozzo) ---")
for node, deg in top_in_nodes:
    print(f"  {node}: {deg} archi entranti")

print("\n--- Top 5 Nodi per Out-Degree (Sorgente) ---")
for node, deg in top_out_nodes:
    print(f"  {node}: {deg} archi uscenti")

def get_in_ego_network(G, node):
    """Ottiene l'ego network entrante (predecessori del nodo)"""
    predecessors = list(G.predecessors(node))
    nodes = [node] + predecessors
    return G.subgraph(nodes).copy()

def get_out_ego_network(G, node):
    """Ottiene l'ego network uscente (successori del nodo)"""
    successors = list(G.successors(node))
    nodes = [node] + successors
    return G.subgraph(nodes).copy()

# Analisi dettagliata per i top nodi
print("\n--- Analisi Dettagliata Ego Network ---")

# Ego network per il nodo con più in-degree
top_in_node = top_in_nodes[0][0]
in_ego = get_in_ego_network(G, top_in_node)
print(f"\nIn-Ego Network di '{top_in_node}':")
print(f"  Nodi: {in_ego.number_of_nodes()}")
print(f"  Archi: {in_ego.number_of_edges()}")
print(f"  Predecessori: {list(G.predecessors(top_in_node))}")

# Ego network per il nodo con più out-degree
top_out_node = top_out_nodes[0][0]
out_ego = get_out_ego_network(G, top_out_node)
print(f"\nOut-Ego Network di '{top_out_node}':")
print(f"  Nodi: {out_ego.number_of_nodes()}")
print(f"  Archi: {out_ego.number_of_edges()}")
print(f"  Successori: {list(G.successors(top_out_node))}")

# Ego network standard (non diretto, raggio 1)
print(f"\n--- Ego Network Standard ---")
for node, _ in top_in_nodes[:3]:
    ego = nx.ego_graph(G, node, radius=1)
    print(f"  Ego di '{node}': {ego.number_of_nodes()} nodi, {ego.number_of_edges()} archi")

# Visualizzazione grafica
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Visualizza In-Ego Network per top 3 nodi
for idx, (node, deg) in enumerate(top_in_nodes[:3]):
    ax = axes[0, idx]
    in_ego = get_in_ego_network(G, node)
    pos = nx.spring_layout(in_ego, seed=42)
    
    node_colors = ['red' if n == node else 'lightblue' for n in in_ego.nodes()]
    node_sizes = [1000 if n == node else 500 for n in in_ego.nodes()]
    
    nx.draw_networkx_nodes(in_ego, pos, node_color=node_colors, node_size=node_sizes, ax=ax)
    nx.draw_networkx_edges(in_ego, pos, edge_color='gray', arrows=True, 
                           arrowsize=15, ax=ax)
    nx.draw_networkx_labels(in_ego, pos, font_size=8, ax=ax)
    ax.set_title(f'In-Ego: {node}\n({deg} archi entranti)')
    ax.axis('off')

# Visualizza Out-Ego Network per top 3 nodi
for idx, (node, deg) in enumerate(top_out_nodes[:3]):
    ax = axes[1, idx]
    out_ego = get_out_ego_network(G, node)
    pos = nx.spring_layout(out_ego, seed=42)
    
    node_colors = ['green' if n == node else 'lightyellow' for n in out_ego.nodes()]
    node_sizes = [1000 if n == node else 500 for n in out_ego.nodes()]
    
    nx.draw_networkx_nodes(out_ego, pos, node_color=node_colors, node_size=node_sizes, ax=ax)
    nx.draw_networkx_edges(out_ego, pos, edge_color='gray', arrows=True,
                           arrowsize=15, ax=ax)
    nx.draw_networkx_labels(out_ego, pos, font_size=8, ax=ax)
    ax.set_title(f'Out-Ego: {node}\n({deg} archi uscenti)')
    ax.axis('off')

plt.tight_layout()
plt.savefig(f"{output_folder}/ego_network_analysis.png", dpi=150, bbox_inches='tight')
plt.show()

# Tabella riepilogativa
print("\n--- Tabella Riepilogativa Ego Network ---")
ego_data = []
for node in G.nodes():
    in_ego = get_in_ego_network(G, node)
    out_ego = get_out_ego_network(G, node)
    ego_data.append({
        'Nodo': node,
        'In-Degree': in_degrees[node],
        'Out-Degree': out_degrees[node],
        'In-Ego Nodi': in_ego.number_of_nodes(),
        'Out-Ego Nodi': out_ego.number_of_nodes()
    })

df_ego = pd.DataFrame(ego_data)
df_ego = df_ego.sort_values('In-Degree', ascending=False)
print(df_ego.head(15).to_string(index=False))

In [ ]:
# Cell 8: Analisi di Community (Louvain e Girvan-Newman) - Pozzo e Sorgente
print("=" * 60)
print("ANALISI DI COMMUNITY DETECTION")
print("=" * 60)

# Convertiamo in grafo non diretto per alcuni algoritmi
G_undirected = G.to_undirected()

# ==========================================
# LOUVAIN ALGORITHM
# ==========================================
print("\n" + "=" * 40)
print("ALGORITMO LOUVAIN")
print("=" * 40)

# Louvain su grafo non diretto
louvain_partition = community_louvain.best_partition(G_undirected, weight='weight')
louvain_communities = {}
for node, comm in louvain_partition.items():
    if comm not in louvain_communities:
        louvain_communities[comm] = []
    louvain_communities[comm].append(node)

# Calcolo modularità Louvain
louvain_modularity = community_louvain.modularity(louvain_partition, G_undirected, weight='weight')

print(f"\n--- Risultati Louvain ---")
print(f"Numero di community: {len(louvain_communities)}")
print(f"Modularità: {louvain_modularity:.4f}")

print(f"\nDettaglio Community Louvain:")
for comm_id, members in sorted(louvain_communities.items()):
    print(f"  Community {comm_id}: {len(members)} nodi")
    print(f"    Membri: {members}")

# Analisi Pozzo (nodi con alto in-degree)
print(f"\n--- Analisi POZZO (Target) per Community Louvain ---")
in_degrees = dict(G.in_degree())
for comm_id, members in sorted(louvain_communities.items()):
    sink_nodes = [(n, in_degrees[n]) for n in members]
    sink_nodes.sort(key=lambda x: x[1], reverse=True)
    top_sink = sink_nodes[0] if sink_nodes else ('N/A', 0)
    total_in = sum([d for _, d in sink_nodes])
    print(f"  Community {comm_id}: Top Pozzo = {top_sink[0]} ({top_sink[1]} in), Tot In-Degree = {total_in}")

# Analisi Sorgente (nodi con alto out-degree)
print(f"\n--- Analisi SORGENTE (Attacker) per Community Louvain ---")
out_degrees = dict(G.out_degree())
for comm_id, members in sorted(louvain_communities.items()):
    source_nodes = [(n, out_degrees[n]) for n in members]
    source_nodes.sort(key=lambda x: x[1], reverse=True)
    top_source = source_nodes[0] if source_nodes else ('N/A', 0)
    total_out = sum([d for _, d in source_nodes])
    print(f"  Community {comm_id}: Top Sorgente = {top_source[0]} ({top_source[1]} out), Tot Out-Degree = {total_out}")

# ==========================================
# GIRVAN-NEWMAN ALGORITHM
# ==========================================
print("\n" + "=" * 40)
print("ALGORITMO GIRVAN-NEWMAN")
print("=" * 40)

# Girvan-Newman (prendiamo le prime iterazioni)
gn_generator = girvan_newman(G_undirected)

# Trova il numero ottimale di community basato sulla modularità
best_gn_communities = None
best_gn_modularity = -1
gn_results = []

for i, communities in enumerate(gn_generator):
    if i > 10:  # Limitiamo a 10 iterazioni
        break
    communities_list = [set(c) for c in communities]
    mod = modularity(G_undirected, communities_list)
    gn_results.append((len(communities), mod, communities_list))
    if mod > best_gn_modularity:
        best_gn_modularity = mod
        best_gn_communities = communities_list

print(f"\n--- Evoluzione Girvan-Newman ---")
for n_comm, mod, _ in gn_results:
    print(f"  {n_comm} community: Modularità = {mod:.4f}")

print(f"\n--- Risultati Ottimali Girvan-Newman ---")
print(f"Numero ottimale di community: {len(best_gn_communities)}")
print(f"Modularità ottimale: {best_gn_modularity:.4f}")

print(f"\nDettaglio Community Girvan-Newman:")
for idx, members in enumerate(best_gn_communities):
    print(f"  Community {idx}: {len(members)} nodi")
    print(f"    Membri: {list(members)}")

# Analisi Pozzo per Girvan-Newman
print(f"\n--- Analisi POZZO (Target) per Community Girvan-Newman ---")
for idx, members in enumerate(best_gn_communities):
    sink_nodes = [(n, in_degrees[n]) for n in members]
    sink_nodes.sort(key=lambda x: x[1], reverse=True)
    top_sink = sink_nodes[0] if sink_nodes else ('N/A', 0)
    total_in = sum([d for _, d in sink_nodes])
    print(f"  Community {idx}: Top Pozzo = {top_sink[0]} ({top_sink[1]} in), Tot In-Degree = {total_in}")

# Analisi Sorgente per Girvan-Newman
print(f"\n--- Analisi SORGENTE (Attacker) per Community Girvan-Newman ---")
for idx, members in enumerate(best_gn_communities):
    source_nodes = [(n, out_degrees[n]) for n in members]
    source_nodes.sort(key=lambda x: x[1], reverse=True)
    top_source = source_nodes[0] if source_nodes else ('N/A', 0)
    total_out = sum([d for _, d in source_nodes])
    print(f"  Community {idx}: Top Sorgente = {top_source[0]} ({top_source[1]} out), Tot Out-Degree = {total_out}")

# ==========================================
# CONFRONTO RISULTATI
# ==========================================
print("\n" + "=" * 40)
print("CONFRONTO LOUVAIN vs GIRVAN-NEWMAN")
print("=" * 40)

comparison_data = {
    'Metrica': ['Numero Community', 'Modularità', 'Min Nodi/Community', 'Max Nodi/Community', 'Media Nodi/Community'],
    'Louvain': [
        len(louvain_communities),
        f"{louvain_modularity:.4f}",
        min(len(m) for m in louvain_communities.values()),
        max(len(m) for m in louvain_communities.values()),
        f"{np.mean([len(m) for m in louvain_communities.values()]):.2f}"
    ],
    'Girvan-Newman': [
        len(best_gn_communities),
        f"{best_gn_modularity:.4f}",
        min(len(m) for m in best_gn_communities),
        max(len(m) for m in best_gn_communities),
        f"{np.mean([len(m) for m in best_gn_communities]):.2f}"
    ]
}
df_comparison = pd.DataFrame(comparison_data)
print(df_comparison.to_string(index=False))

# ==========================================
# VISUALIZZAZIONE GRAFICA
# ==========================================
fig, axes = plt.subplots(2, 3, figsize=(20, 14))

pos = nx.spring_layout(G_undirected, seed=42, k=2)

# Grafico 1: Community Louvain
ax1 = axes[0, 0]
node_colors_louvain = [louvain_partition[node] for node in G_undirected.nodes()]
nodes = nx.draw_networkx_nodes(G_undirected, pos, node_color=node_colors_louvain, 
                                cmap=plt.cm.Set1, node_size=500, ax=ax1)
nx.draw_networkx_edges(G_undirected, pos, alpha=0.3, ax=ax1)
nx.draw_networkx_labels(G_undirected, pos, font_size=6, ax=ax1)
ax1.set_title(f'Community Louvain\n({len(louvain_communities)} community, mod={louvain_modularity:.3f})')
ax1.axis('off')

# Grafico 2: Community Girvan-Newman
ax2 = axes[0, 1]
gn_partition = {}
for idx, members in enumerate(best_gn_communities):
    for node in members:
        gn_partition[node] = idx
node_colors_gn = [gn_partition[node] for node in G_undirected.nodes()]
nodes = nx.draw_networkx_nodes(G_undirected, pos, node_color=node_colors_gn,
                                cmap=plt.cm.Set2, node_size=500, ax=ax2)
nx.draw_networkx_edges(G_undirected, pos, alpha=0.3, ax=ax2)
nx.draw_networkx_labels(G_undirected, pos, font_size=6, ax=ax2)
ax2.set_title(f'Community Girvan-Newman\n({len(best_gn_communities)} community, mod={best_gn_modularity:.3f})')
ax2.axis('off')

# Grafico 3: Confronto Modularità GN evolution
ax3 = axes[0, 2]
n_communities = [r[0] for r in gn_results]
modularities = [r[1] for r in gn_results]
ax3.plot(n_communities, modularities, 'bo-', linewidth=2, markersize=8)
ax3.axhline(louvain_modularity, color='red', linestyle='--', label=f'Louvain ({louvain_modularity:.3f})')
best_idx = modularities.index(max(modularities))
ax3.scatter([n_communities[best_idx]], [modularities[best_idx]], color='green', s=200, zorder=5, label='Best GN')
ax3.set_xlabel('Numero di Community')
ax3.set_ylabel('Modularità')
ax3.set_title('Evoluzione Modularità Girvan-Newman')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Grafico 4: Dimensione Community Louvain
ax4 = axes[1, 0]
louvain_sizes = [len(members) for members in louvain_communities.values()]
ax4.bar(range(len(louvain_sizes)), louvain_sizes, color=plt.cm.Set1(np.linspace(0, 1, len(louvain_sizes))))
ax4.set_xlabel('Community ID')
ax4.set_ylabel('Numero di Nodi')
ax4.set_title('Dimensione Community - Louvain')
ax4.set_xticks(range(len(louvain_sizes)))

# Grafico 5: Dimensione Community Girvan-Newman
ax5 = axes[1, 1]
gn_sizes = [len(members) for members in best_gn_communities]
ax5.bar(range(len(gn_sizes)), gn_sizes, color=plt.cm.Set2(np.linspace(0, 1, len(gn_sizes))))
ax5.set_xlabel('Community ID')
ax5.set_ylabel('Numero di Nodi')
ax5.set_title('Dimensione Community - Girvan-Newman')
ax5.set_xticks(range(len(gn_sizes)))

# Grafico 6: Heatmap confronto
ax6 = axes[1, 2]
# Creiamo una matrice di sovrapposizione tra le community
overlap_matrix = np.zeros((len(louvain_communities), len(best_gn_communities)))
for i, l_members in enumerate(louvain_communities.values()):
    for j, gn_members in enumerate(best_gn_communities):
        overlap = len(set(l_members) & set(gn_members))
        overlap_matrix[i, j] = overlap

sns.heatmap(overlap_matrix, annot=True, fmt='.0f', cmap='YlOrRd', ax=ax6,
            xticklabels=[f'GN_{i}' for i in range(len(best_gn_communities))],
            yticklabels=[f'L_{i}' for i in range(len(louvain_communities))])
ax6.set_xlabel('Community Girvan-Newman')
ax6.set_ylabel('Community Louvain')
ax6.set_title('Sovrapposizione tra Community')

plt.tight_layout()
plt.savefig(f"{output_folder}/community_analysis.png", dpi=150, bbox_inches='tight')
plt.show()

# Tabella finale riepilogativa
print("\n" + "=" * 60)
print("RIEPILOGO FINALE")
print("=" * 60)
print(f"\nMiglior algoritmo per modularità: {'Louvain' if louvain_modularity > best_gn_modularity else 'Girvan-Newman'}")
print(f"Louvain Modularity: {louvain_modularity:.4f}")
print(f"Girvan-Newman Modularity: {best_gn_modularity:.4f}")